In [21]:
from assets.database import db_session
from assets.models import Data

from keras.models import Sequential
import numpy as np
import pandas as pd
import requests
import pickle
from datetime import datetime
from datetime import timedelta

In [6]:
# DBからデータを取得
data = db_session.query(Data.date, Data.real_price, Data.pred_price).all()

# 取得したデータを項目ごとに分配
dates = []
real_prices = []
pred_prices = []

for _data in data:
    dates.append(_data.date)
    real_prices.append(_data.real_price)
    pred_prices.append(_data.pred_price)
    
dates = np.array(dates)
real_prices = np.array(real_prices)
pred_prices = np.array(pred_prices)

real_prices

2021-03-13 15:56:27,594 INFO sqlalchemy.engine.base.Engine SELECT data.date AS data_date, data.real_price AS data_real_price, data.pred_price AS data_pred_price 
FROM data
2021-03-13 15:56:27,599 INFO sqlalchemy.engine.base.Engine ()


array([376384.0, 369767.0, 368506.0, 396319.0, 395754.0, 398468.0,
       394797.0, 395085.0, 395099.0, 392999.0, 393923.0, 395229.0,
       400000.0, 427531.0, 429902.0, 435816.0, 432185.0, 435789.0,
       453900.0, 413712.0, 423575.0, 419629.0, 421267.0, 422865.0,
       425943.0, 426039.0, 423661.0, 413871.0, 429450.0, 429848.0,
       430100.0, 427484.0, 435287.0, 433206.0, 428795.0, 429580.0,
       428300.0, 430167.0, 434178.0, 443635.0, 441454.0, 442176.0,
       445064.0, 445814.0, 440825.0, 438504.0, 438500.0, 436971.0,
       429928.0, 433151.0, 444477.0, 443222.0, 452501.0, 453000.0,
       453417.0, 460500.0, 545469.0, 555300.0, 548000.0, 562532.0,
       563489.0, 579725.0, 588520.0, 576430.0, 591143.0, 564000.0,
       569500.0, 568149.0, 577750.0, 563500.0, 581653.0, 584418.0,
       591395.0, 591437.0, 595646.0, 594101.0, 603102.0, 619737.0,
       611479.0, 579000.0, 579700.0, 579990.0, 579000.0, 577863.0,
       587393.0, 593000.0, 601458.0, 628473.0, 640290.0, 63287

In [24]:
# DBからデータを取得
data = db_session.query(Data.date, Data.real_price, Data.pred_price).all()

# 取得したデータを項目ごとに分配
dates = []
real_prices = []
pred_prices = []

for _data in data:
    dates.append(_data.date)
    real_prices.append(_data.real_price)
    pred_prices.append(_data.pred_price)
    
df = pd.DataFrame({'date':dates, 'real_price':real_prices, 'pred_price':pred_prices})
df.tail()

2021-03-13 17:05:29,942 INFO sqlalchemy.engine.base.Engine SELECT data.date AS data_date, data.real_price AS data_real_price, data.pred_price AS data_pred_price 
FROM data
2021-03-13 17:05:29,946 INFO sqlalchemy.engine.base.Engine ()


,date,real_price,pred_price
746,2021-02-21 09:00:00,5910115.0,5.998294e+06
747,2021-02-22 09:00:00,6080178.0,6.000729e+06
748,2021-02-23 09:00:00,5687069.0,6.152566e+06
749,2021-02-24 09:00:00,5422284.0,5.721341e+06
750,2021-02-25 09:00:00,NaN,5.521486e+06


In [26]:
df['real_price'][:-1].tail(10)

740    5111984.0
741    5055269.0
742    5205481.0
743    5526975.0
744    5456000.0
745    5910646.0
746    5910115.0
747    6080178.0
748    5687069.0
749    5422284.0
Name: real_price, dtype: float64

In [27]:
df['5ma'] = df['real_price'].rolling(5, min_periods=1).mean()
df.tail(10)

,date,real_price,pred_price,5ma
741,2021-02-16 09:00:00,5055269.0,5.140096e+06,5022612.6
742,2021-02-17 09:00:00,5205481.0,5.076019e+06,5060767.4
743,2021-02-18 09:00:00,5526975.0,5.234057e+06,5171420.6
744,2021-02-19 09:00:00,5456000.0,5.605767e+06,5271141.8
745,2021-02-20 09:00:00,5910646.0,5.514918e+06,5430874.2
746,2021-02-21 09:00:00,5910115.0,5.998294e+06,5601843.4
747,2021-02-22 09:00:00,6080178.0,6.000729e+06,5776782.8
748,2021-02-23 09:00:00,5687069.0,6.152566e+06,5808801.6
749,2021-02-24 09:00:00,5422284.0,5.721341e+06,5802058.4
750,2021-02-25 09:00:00,NaN,5.521486e+06,5774911.5


In [10]:
type(real_prices[1])

float

In [19]:
period = 5
moving_averages = np.convolve(real_prices[:-1], np.ones(period)/float(period), 'same')
moving_averages

array([381346.0, 385762.80000000005, 390768.80000000005,
       396084.60000000003, 395840.60000000003, 395289.6, 394380.6,
       394466.99999999994, 395450.0, 401936.4, 409317.00000000006,
       417695.60000000003, 425086.80000000005, 432244.60000000003,
       437518.4, 434280.4, 431832.2, 429321.0, 426416.60000000003,
       420209.60000000003, 422655.80000000005, 423148.60000000003,
       423955.00000000006, 422475.80000000005, 423792.80000000005,
       424573.80000000005, 425386.0, 426150.60000000003,
       430433.80000000005, 431185.00000000006, 430974.4, 430870.4,
       431033.60000000003, 430009.60000000003, 430204.0, 433172.0,
       435546.8, 438322.0, 441301.4, 443628.6, 443066.6, 442476.6,
       441741.39999999997, 440122.8, 436945.6, 435410.8, 436605.4,
       437549.80000000005, 440655.80000000005, 445270.20000000007,
       449323.4, 452528.00000000006, 472977.4, 493537.2, 512537.2,
       534360.2, 554958.0, 561809.2, 568453.2, 574139.2, 579861.4,
       579963.6

In [15]:
# 移動平均の点数
n = 3
    
    # 配列生成
data = np.array([ 1,  2,  5,  4,  2,  4,  3])
    
    # コンボリューション積分で移動平均の計算
ave = np.convolve(data, np.ones(n)/float(n), 'same')

ave

array([1.        , 2.66666667, 3.66666667, 3.66666667, 3.33333333,
       3.        , 2.33333333])

In [17]:
data[:-1]

array([1, 2, 5, 4, 2, 4])

In [33]:
db_session.query(Data).all()[-1].date

2021-02-22 01:58:04,588 INFO sqlalchemy.engine.base.Engine SELECT data.id AS data_id, data.date AS data_date, data.real_price AS data_real_price, data.pred_price AS data_pred_price, data.timestamp AS data_timestamp 
FROM data
2021-02-22 01:58:04,590 INFO sqlalchemy.engine.base.Engine ()


datetime.datetime(2021, 2, 21, 0, 0)

In [19]:
a = db_session.query(Data).all()[-5:]
input_will = []

for price in a:
    input_will.append(price.real_price)
input_will = np.array(input_will).reshape(1, 5, 1)
input_mean = np.mean(input_will)
input_std = np.std(input_will)
sta_input = (input_will - input_mean) / input_std
sta_input

2021-02-22 01:44:55,385 INFO sqlalchemy.engine.base.Engine SELECT data.id AS data_id, data.date AS data_date, data.real_price AS data_real_price, data.pred_price AS data_pred_price, data.timestamp AS data_timestamp 
FROM data
2021-02-22 01:44:55,388 INFO sqlalchemy.engine.base.Engine ()


array([[[-1.44852977],
        [-0.27361098],
        [-0.53299331],
        [ 1.12853731],
        [ 1.12659674]]])

In [38]:
# 翌日の日付を取得
today = db_session.query(Data).all()[-1].date
next_day = today + timedelta(days=1)
next_day

2021-02-22 02:00:07,729 INFO sqlalchemy.engine.base.Engine SELECT data.id AS data_id, data.date AS data_date, data.real_price AS data_real_price, data.pred_price AS data_pred_price, data.timestamp AS data_timestamp 
FROM data
2021-02-22 02:00:07,732 INFO sqlalchemy.engine.base.Engine ()


datetime.datetime(2021, 2, 22, 0, 0)

In [1]:
from assets.database import db_session
from assets.models import Data

from keras.models import Sequential
import numpy as np
import requests
import pickle
#from datetime import datetime
from datetime import timedelta

def get_price():
    URL = 'https://coincheck.com/api/ticker'
    coincheck = requests.get(URL).json() 
    last_price = coincheck['last']
    
    return last_price

def predict_nextday():

    # 価格情報を取得しデータフレームに格納
    today_data = db_session.query(Data).all()[-1]
    today_data.real_price = get_price()  # 上書き
    db_session.add(today_data)
    db_session.commit()

    # windowを作成し標準化
    window = 5

    data_five = db_session.query(Data).all()[-window:]
    input_will = []

    for price in data_five:
        input_will.append(price.real_price)
    input_will = np.array(input_will).reshape(1, window, 1)
    input_mean = np.mean(input_will)
    input_std = np.std(input_will)
    sta_input = (input_will - input_mean) / input_std
    print(sta_input)

    # 学習済みモデルを取得
    with open("lstm_model.pickle", "rb") as f:
        model = pickle.load(f)

    # 翌日の価格を予測、標準化してあるので元に戻す
    future_price = model.predict(sta_input)
    future_price = future_price.reshape(-1)
    future_price = future_price*input_std + input_mean
    future_price = float(future_price)
    print(future_price)

    # 翌日の日付を取得
    today = db_session.query(Data).all()[-1].date
    next_day = today + timedelta(days=1)
    print(next_day)

    # 翌日の日付、予測した価格をデータベースに格納
    row = Data(date = next_day, real_price = None, pred_price = future_price)
    db_session.add(row)
    db_session.commit()

Using TensorFlow backend.
C:\Users\tomot\anaconda3\envs\lstm\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomot\anaconda3\envs\lstm\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomot\anaconda3\envs\lstm\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomot\anaconda3\envs\lstm\lib\site-packag

In [2]:
predict_nextday()

2021-02-23 17:13:06,304 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-23 17:13:06,311 INFO sqlalchemy.engine.base.Engine ()
2021-02-23 17:13:06,321 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-23 17:13:06,324 INFO sqlalchemy.engine.base.Engine ()
2021-02-23 17:13:06,331 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-23 17:13:06,344 INFO sqlalchemy.engine.base.Engine SELECT data.id AS data_id, data.date AS data_date, data.real_price AS data_real_price, data.pred_price AS data_pred_price, data.timestamp AS data_timestamp 
FROM data
2021-02-23 17:13:06,349 INFO sqlalchemy.engine.base.Engine ()
2021-02-23 17:13:08,238 INFO sqlalchemy.engine.base.Engine UPDATE data SET real_price=? WHERE data.id = ?
2021-02-23 17:13:08,243 INFO sqlalchemy.engine.base.Engine (5129755.0, 749)
2021-02-23 17:13:08,256 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-23 17:13:08,291 IN

In [47]:
# 翌日の日付、予測した価格をデータベースに格納
row = Data(date = next_day, real_price = None, pred_price = future_price)
db_session.add(row)
db_session.commit()

2021-02-22 02:09:53,807 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 02:09:53,809 INFO sqlalchemy.engine.base.Engine ('2021-02-22 00:00:00.000000', None, 6211567.0, '2021-02-22 01:09:37.634966')
2021-02-22 02:09:53,818 INFO sqlalchemy.engine.base.Engine COMMIT


In [51]:
print(db_session.query(Data).all()[-1].real_price)

2021-02-22 02:11:03,158 INFO sqlalchemy.engine.base.Engine SELECT data.id AS data_id, data.date AS data_date, data.real_price AS data_real_price, data.pred_price AS data_pred_price, data.timestamp AS data_timestamp 
FROM data
2021-02-22 02:11:03,158 INFO sqlalchemy.engine.base.Engine ()
None
